# EDA

In [324]:
import numpy as np
import pandas as pd 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [325]:
df = pd.read_csv("data/personal_transactions.csv")
df.Date = pd.to_datetime(df.Date)

In [326]:
df.head()

,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card


In [327]:
df.shape

(806, 6)

In [328]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              806 non-null    datetime64[ns]
 1   Description       806 non-null    object        
 2   Amount            806 non-null    float64       
 3   Transaction Type  806 non-null    object        
 4   Category          806 non-null    object        
 5   Account Name      806 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 37.9+ KB


In [329]:
df.describe()

,Amount
count,806.000000
mean,273.391489
std,667.630374
min,1.750000
25%,15.687500
50%,37.480000
75%,117.680000
max,9200.000000


In [330]:
def count_sum(data ,column: str, plot = "Pie"):
    by_column = data\
                .groupby(column)\
                .agg({"Transaction Type": "count", "Amount": "sum"})\
                .rename(columns={"Transaction Type": "Total"})\
                .reset_index()
    by_column.columns = [column, "Total", "Sum"]
    
    labels = by_column[column]

    fig = None
    
    if plot == 'Pie':
        fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Sum"],
                name="Sum"
            ),
        1, 1)
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Total"],
                name="Total"
            ),
        1, 2)
    elif plot == 'Scatter':
        fig = make_subplots(1, 2,
                    subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Sum"],
                name="Sum"
            ),
        1, 1)
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Total"],
                name="Total"
            ),
        1, 2)
        

    fig.update_layout(title_text=f"{column} Analysis")
    fig.show()
    
    return by_column

#### Debit Analysis

In [331]:
debits = df[df["Transaction Type"] == 'debit']
gastos_by_cat = count_sum(debits, "Category")
gastos_by_cat

,Category,Total,Sum
0,Alcohol & Bars,25,539.13
1,Auto Insurance,18,1350.00
2,Coffee Shops,31,115.54
3,Credit Card Payment,71,33041.36
4,Electronics & Software,4,719.00
5,Entertainment,1,9.62
6,Fast Food,16,330.63
7,Food & Dining,2,77.75
8,Gas & Fuel,52,1715.17
9,Groceries,105,2795.21


#### Payment Category Destribution

In [332]:
df_category=df['Category'].value_counts().to_frame().reset_index().rename(columns={'index':'Category','Category':'count'})

fig = go.Figure(go.Bar(
    x=df_category['Category'],y=df_category['count'],
    marker={'color': df_category['count'], 
    'colorscale': 'Viridis'},  
    text=df_category['count'],
    textposition = "outside",
))
fig.update_layout(title_text='Category Distribution',xaxis_title="Category",yaxis_title="Count",title_x=0.5)
fig.show()

#### Transaction Type Distribution

In [333]:
df_trans=df['Transaction Type'].value_counts().to_frame().reset_index().rename(columns={'index':'Transaction Type','Transaction Type':'count'})

fig = go.Figure(go.Bar(
    x=df_trans['Transaction Type'],y=df_category['count'],
    marker={'color': df_category['count'], 
    'colorscale': 'Viridis'},  
    text=df_category['count'],
    textposition = "outside",
))
fig.update_layout(title_text='Transaction Type Distribution',xaxis_title="Category",yaxis_title="Count",title_x=0.5)
fig.show()

#### Missing Values

In [334]:
#null
df.isnull().sum()

Date                0
Description         0
Amount              0
Transaction Type    0
Category            0
Account Name        0
dtype: int64

In [335]:
#find the percentage of all missing values
features_with_null_values=[features for features in df.columns if df[features].isnull().sum()>1]

for feature in features_with_null_values:
    print(feature," : ",np.round(df[feature].isnull().mean(),4),"%")
          
print("\nfeature count with null values ",len(features_with_null_values))


feature count with null values  0


# Encoding

In [336]:
target_name = "Amount"
target = df[target_name]

data = df.drop(columns=[target_name, "Date"], axis=1)

#### identify categorical variables

In [337]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
categorical_columns

['Description', 'Transaction Type', 'Category', 'Account Name']

In [338]:
data_categorical = data[categorical_columns]
data_categorical.head()

,Description,Transaction Type,Category,Account Name
0,Amazon,debit,Shopping,Platinum Card
1,Mortgage Payment,debit,Mortgage & Rent,Checking
2,Thai Restaurant,debit,Restaurants,Silver Card
3,Credit Card Payment,credit,Credit Card Payment,Platinum Card
4,Netflix,debit,Movies & DVDs,Platinum Card


In [339]:
print(f"The dataset is composed of {data_categorical.shape[1]} features")

The dataset is composed of 4 features


In [340]:
encoder = OneHotEncoder(sparse=False)
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.

In [341]:
print(f"The encoded dataset contains {data_encoded.shape[1]} features")

The encoded dataset contains 92 features


In [342]:
columns_encoded = encoder.get_feature_names_out(data_categorical.columns)
df_encoded=pd.DataFrame(data_encoded, columns=columns_encoded)
df_encoded.head()

,Description_Amazon,Description_Amazon Video,Description_American Tavern,Description_BBQ Restaurant,Description_BP,Description_Bakery Place,Description_Barbershop,Description_Belgian Restaurant,Description_Best Buy,Description_Biweekly Paycheck,...,Category_Movies & DVDs,Category_Music,Category_Paycheck,Category_Restaurants,Category_Shopping,Category_Television,Category_Utilities,Account Name_Checking,Account Name_Platinum Card,Account Name_Silver Card
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [343]:
model = make_pipeline(OneHotEncoder(handle_unknown="ignore"), LinearRegression())

In [344]:
cv_results = cross_validate(model, data_categorical, target)
cv_results

{'fit_time': array([0.08499789, 0.07982731, 0.07579756, 0.07737947, 0.0531354 ]),
 'score_time': array([0.00600028, 0.01019549, 0.00500703, 0.00599241, 0.0079577 ]),
 'test_score': array([0.85605636, 0.95305402, 0.95721573, 0.90100143, 0.92874029])}

# Train Model and Evaluation

In [345]:
def performance_metric(y_true, y_predict):    
    # Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    # Return the score
    return score

In [346]:
model=LinearRegression()

In [347]:
x = df_encoded
y = target

In [348]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [349]:
model.fit(X_train,y_train)

LinearRegression()

In [350]:
r_sq=model.score(X_train,y_train)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

coefficient of determination: 0.934450072084785
intercept: 451620341126536.9
slope: [ 2.73676755e+12  1.26684175e+16 -3.88046317e+15 -3.88046317e+15
 -1.57410133e+14 -3.88046317e+15  6.07014595e+15 -3.88046317e+15
 -5.21317847e+15  2.30641152e+16 -1.91336325e+16 -3.88046317e+15
 -3.88046317e+15 -3.88046317e+15 -1.57410133e+14 -3.88046317e+15
 -3.88046317e+15 -1.57410133e+14 -3.35054439e+16 -1.57410133e+14
  7.12616707e+15 -1.57410133e+14 -3.88046317e+15 -1.91336325e+16
 -3.35054439e+16 -1.57410133e+14 -3.88046317e+15 -1.57410133e+14
 -3.88046317e+15 -1.91336325e+16 -7.10751088e+15  7.21888872e+14
  7.73438976e+14 -3.88046317e+15 -3.88046317e+15 -3.88046317e+15
 -3.88046317e+15 -3.88046317e+15 -3.88046317e+15 -3.88046317e+15
 -3.88046317e+15  5.91814218e+14  3.14727557e+15  1.26684175e+16
  1.26684175e+16 -3.88046317e+15  1.08062719e+16 -3.88046317e+15
 -3.35054439e+16 -1.57410133e+14 -3.88046317e+15 -3.88046317e+15
  4.48198142e+14 -1.57410133e+14  2.86234630e+15 -3.88046317e+15
  1.31

In [351]:
predicted = model.predict(X_test)

In [352]:
d={"actual":y_test,"predicted":predicted}
df_result=pd.DataFrame(data=d)
df_result

,actual,predicted
644,8.00,3.887500e+01
645,36.76,7.187500e+01
646,30.99,6.787500e+01
647,34.33,1.228750e+02
648,765.68,4.508750e+02
...,...,...
801,2250.00,1.960875e+03
802,33.46,7.087500e+01
803,4.27,6.056083e+14
804,1.75,-1.250000e-01


In [353]:
performance_metric(y_test,predicted)

-6.296125321602128e+23

In [354]:
import plotly.express as px
fig = px.scatter(df_result, x="actual", y="predicted")
fig.show()

# Decision Tree Regressor

In [355]:
# Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(df_encoded, target, test_size=0.2, random_state=0)

# Success
print("Training and testing split was successful.")

Training and testing split was successful.


In [356]:
def fit_model(X, y):    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)
    cv_sets.get_n_splits(X)

    # Create a decision tree regressor object
    regressor = DecisionTreeRegressor(random_state=0)

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':range(1,11)}

    # Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid

In [357]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(reg.best_estimator_.get_params()['max_depth']))
print("Best Score is {:.2f}".format(reg.best_score_))

Parameter 'max_depth' is 8 for the optimal model.
Best Score is 0.68


In [358]:
reg=DecisionTreeRegressor(random_state=0, max_depth=8)
reg.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=8, random_state=0)

In [359]:
predicted = reg.predict(X_test)

In [360]:
d={"actual":y_test,"predicted":predicted}
df_result=pd.DataFrame(data=d)
df_result

,actual,predicted
8,15.38,55.296364
258,36.24,27.247201
172,8000.00,9200.000000
317,11.76,27.247201
294,5.64,27.247201
...,...,...
255,30.00,36.473684
162,20.52,27.247201
116,6.41,27.247201
37,11.76,27.247201


In [361]:
import plotly.express as px
fig = px.scatter(df_result, x="actual", y="predicted")
fig.show()